In [ ]:
#Задание №1

def qZ(x, y):
    return (x - 2 * y) / (x * x + y * y + 1)

def qSumZ(Z):
    return sum(Z)

#Процесс обмена хромосомами
def exchangeScheme(oldX, oldY, sortedId):
    X = [0 for i in range(4)]
    Y = [0 for i in range(4)]    
    X[2] = oldX[sortedId[2]]
    X[3] = oldX[sortedId[2]]
    X[0] = oldX[sortedId[0]]
    X[1] = oldX[sortedId[1]]   
    Y[0] = oldX[sortedId[2]]
    Y[1] = oldX[sortedId[2]]
    Y[2] = oldX[sortedId[0]]
    Y[3] = oldX[sortedId[1]]    
    return X, Y

#Сортировка качества потомков
def sorting(Z):
    sortedId = sorted(range(len(Z)), key = lambda k: Z[k])
    return sortedId

#Шаг эволюции
def evoStep(X, Y, Z):
    _, minId = min((value, id) for (id, value) in enumerate(Z))
    X = X[:]
    Y = Y[:]
    Z = Z[:]
    X.pop(minId)
    Y.pop(minId)
    Z.pop(minId)
    return X, Y, Z

#Шаги эволюции
def evoSteps(X, Y, stepsNum = 4):
    results = []
    for i in range(4):
        arrZ = [qZ(x, Y[i]) for i, x in enumerate(X)]
        X, Y, Z = evoStep(X, Y, arrZ)
        X, Y = exchangeScheme(X, Y, sorting(Z))
        results.append([X, Y, qSumZ(arrZ), arrZ])
    return X, Y, results

#Входные параметры
X = [-1, 0, 2, 3]
Y = [0, -1, -2, 1]

results = evoSteps(X, Y)

for i in range(len(results[2])):
    print(f'max_{i + 1}_step: {results[2][i][2]}')

qualityArrZ = []
for i in range(len(results[2])):
    qualityArrZ += results[2][i][3]
    
print(f'max Z:      {max(qualityArrZ)}')

In [ ]:
#Задача №2

import networkx as nx
from math import e

#Входные параметры
distances = [(1, 2, 19), (1, 3, 39), (1, 4, 35), (1, 5, 26), (1, 6, 18), (2, 3, 26), (2, 4, 33), (2, 5, 37), (2, 6, 14), (3, 4, 22), (3, 5, 41), (3, 6, 21), (4, 5, 22), (4, 6, 19), (5, 6, 24)]
V = [1, 4, 2, 3, 5, 6, 1]
Z = [(5, 2), (4, 5), (2, 3), (3, 4)]
P = [88, 54, 24, 64]
T = 100

#Функция вероятности
def probability(delta, T):
    return 100 * e ** (-delta / T)

#Функция изменения температуры
def reductTemp(prevT):
    nextT = 0.5 * prevT
    return nextT

#Построение начального графа
graph = nx.Graph()
graph.add_weighted_edges_from(distances)
nx.draw_kamada_kawai(graph, node_color = '#2ce660', node_size = 2000, with_labels = True)

#Функция вычисления длины ребра
def edgeLength(i , j, distances, roundTrip = True):
    if roundTrip:
        return max([(item[2] if (item[0] == i and item[1] ==j) or (item[1] == i and item[0] == j) else -1) for item in distances])
    else:
        return max([(item[2] if (item[0] == i and item[1] == j) else -1) for item in distances])

#Функция вычисления длины маршрута
def routeLength(V, distances):
    edges = []
    for i in range(len(V) - 1):
        edges.append(edgeLength(V[i], V[i + 1], distances))
    return sum(edges)

#Функция перестановки в пути
def routeOneReplacement(arrV, Z, replacementByName = True):
    decrement = 1 if replacementByName else 0
    arrV[Z[0] - decrement], arrV[Z[1] - decrement] = arrV[Z[1] - decrement], arrV[Z[0] - decrement]
    return arrV

#Функция реализации перестановки в пути
def routeReplacement(V, Z):
    for z in Z:
        V = routeOneReplacement(V, z)
    return V

#Функция нужного пути методом отжига
def chooseRoute(distances, V, Z, T, P):
    sumLength = routeLength(V, distances)
    arrSum = [sumLength]
    
    for i in range(len(Z)):
        newV = routeOneReplacement(V[:], Z[i])
        newS = routeLength(newV, distances)
        arrSum.append(newS)
        deltaS = newS - sumLength
        if deltaS > 0:
            p = probability(deltaS, T)
            if p > P[i]:
                V = newV
                sumLength = newS
        else:
            V = newV
            sumLength = newS
        T = reductTemp(T)
    return V, arrSum

#Построение конечного графа
def drawRouteGraph(distances, bestRoute):
    newDistances = []
    for i in range(len(bestRoute) - 1):
        for distance in distances:
            if (distances[0] == bestRoute[i] and distances[1] == bestRoute[i + 1]) or (distances[1] == bestRoute[i] and distances[0] == bestRoute[i + 1]):
                newDistances.append(distance)
    graph = nx.Graph()
    graph.add_weighted_edges_from(newDistances)
    nx.draw_kamada_kawai(graph, node_color = '#e1eb26', node_size = 2000, with_labels = True)

#Информация о лучшем маршруте
bestRoute, arrLength = chooseRoute(distances, V, Z, T, P)
print(f'Лучший выбранный маршрут: {bestRoute}')
print(f'Длина лучшего выбранного маршрута: {routeLength(bestRoute, distances)}')
print(f'Длины всех рассмотренных маршрутов: {arrLength}')
drawRouteGraph(distances, bestRoute)